## Assigment 7: Concurrency

__Website uptime__ is the time that a website or web service is available to the users over a given period.

The task is to build an application that checks the uptime of websites. 

- The application should go over a list of website URLs and checks if those websites are up.
- Instead of performing a classic HTTP GET request, it performs a HEAD request so that it does not affect traffic significantly.
- If the HTTP status is in the danger ranges (400+, 500+), a message is casted. 

Here are some useful functions:

In [ ]:
#### _website uptimer_ ####

import time
import logging
import requests
from multiprocessing.pool import Pool
from queue import Queue
from threading import Thread

 
class WebsiteDownException(Exception):
    pass
 
def ping_website(address, timeout=20):
    """
    Check if a website is down. A website is considered down 
    if either the status_code >= 400 or if the timeout expires
     
    Throw a WebsiteDownException if any of the website down conditions are met
    """
    try:
        response = requests.head(address, timeout=timeout)
        if response.status_code >= 400:
            logging.warning("Website %s returned status_code=%s" % (address, response.status_code))
            raise WebsiteDownException()
    except requests.exceptions.RequestException:
        logging.warning("Timeout expired for website %s" % address)
        raise WebsiteDownException()
         
def check_website(address):
    """
    Utility function: check if a website is down, if so, notify the user
    """
    try:
        ping_website(address)
    except WebsiteDownException:
        print('The website '+address+' is down')

You need a website list to try our system out. Create your own list or use the following one:

In [ ]:
WEBSITE_LIST = [
    'http://envato.com',
    'http://amazon.co.uk',
    'http://amazon.com',
    'http://facebook.com',
    'http://google.com',
    'http://google.fr',
    'http://google.es',
    'http://google.co.uk',
    'http://internet.org',
    'http://gmail.com',
    'http://stackoverflow.com',
    'http://github.com',
    'http://heroku.com',
    'http://really-cool-available-domain.com',
    'http://djangoproject.com',
    'http://rubyonrails.org',
    'http://basecamp.com',
    'http://trello.com',
    'http://yiiframework.com',
    'http://shopify.com',
    'http://another-really-interesting-domain.co',
    'http://airbnb.com',
    'http://instagram.com',
    'http://snapchat.com',
    'http://youtube.com',
    'http://baidu.com',
    'http://yahoo.com',
    'http://live.com',
    'http://linkedin.com',
    'http://yndex.ru',
    'http://netflix.com',
    'http://wordpress.com',
    'http://bing.com',
]

A serial version of the _website uptimer_ can be written as: 

In [ ]:
import time
 
start_time = time.time()
 
for address in WEBSITE_LIST:
    check_website(address)
         
end_time = time.time()        
 
print("Time for Serial: %ssecs" % (end_time - start_time))

You should build two versions of the _website uptimer_, one using threads and another using multiprocessing. Compare the time of each version and write a short explanation of what you are observing.

#  Some websites are down to begin with such as live.com, some others are down because we are using the http versions instead of the https versions.

In [8]:
def multi_processes_check():
    start_time = time.time()
     
    with Pool(8) as p:
        p.map(check_website, WEBSITE_LIST)
    
    end_time = time.time()        
 
    print("Time for Multi-Process time: %ssecs" % (end_time - start_time))

multi_processes_check()

The website http://really-cool-available-domain.com is down


The website http://another-really-interesting-domain.co is down


The website http://netflix.com is down


The website http://live.com is down


The website http://bing.com is down
Time for Multi-Process time: 0.7834949493408203secs


In [9]:
def check_worker(q):
    while True:
        site = q.get()
        check_website(site)
        q.task_done()

def multi_threaded_check():
    
    q = Queue()
    for i in WEBSITE_LIST:
        q.put(i)
    num_threads = 4 
    start_time = time.time()
    for i in range(num_threads):
        worker = Thread(target=check_worker, args=(q,))
        worker.setDaemon(True)
        worker.start()
    
    q.join()
    end_time = time.time()        
 
    print("Time for Multi-Threaded time: %ssecs" % (end_time - start_time))

multi_threaded_check()

The website http://really-cool-available-domain.com is down
The website http://another-really-interesting-domain.co is down
The website http://live.com is down


The website http://netflix.com is down
The website http://bing.com is down
Time for Multi-Threaded time: 1.7526021003723145secs
